In [1]:
import pandas as pd
from graphique import *
import numpy as np
import datetime
from morse3 import Morse as m
import string, random

truth_file_url = "https://drive.usercontent.google.com/download?id=1KE4dJ_ArA7jhIUYmzITIYO7Yh60rQ0-K&export=download&authuser=2&confirm=t&uuid=5b074238-7709-408b-ac93-86cf839cdb07&at=APZUnTXJHY4NiV5TeGbmEtE-F6Ip:1699105244012"

# Charger le fichier de vérité dans un dataframe sans le télécharger
df = pd.read_csv("../truth_ground.csv", delimiter= '\t')    
df.columns = ["id","date", "long", "lat"]
columns_types = {'id' : np.int16, 'date': str, 'long': np.float16, 'lat': np.float16}
df = df.astype(columns_types)

In [2]:
#df_slice = df.loc[(df['id']==1) | (df['id']==2)]
df_slice = df.loc[(df['id']==1)]
#print(df_slice.shape)
#df_slice.head(5)
#draw_map(1, df_slice, "Cartes/heatmap_1")

NameError: name 'df' is not defined

In [ ]:
# Bucketisation en tuiles représentant des quartiers (arrondi 0.01)
df_slice['long'] = df_slice['long'].apply(lambda x : round(x, 2))
df_slice['lat'] = df_slice['lat'].apply(lambda x : round(x, 2))
#draw_map(1, df_slice, "Cartes/heatmap_1_tuiles")

In [4]:
# Découper en tuiles plus petites que les tuiles finales pour pouvoir supprimer des lignes de POI plus tard
# Déterminer les poi pour les garder intègres au maximum
# Décaler les heures de 6h tout en gardant la même durée totale passée dans ces poi -- Distribution normale...
def diff_time(time1, last_date_tab : Dict):
    if last_date_tab[1] is None:
        last_date_tab[1] = time1
        return datetime.timedelta()
    else:
        difference = time1 - last_date_tab[1]
        last_date_tab[1] = time1
        return difference


def computePOI(df_slice_id: pd.DataFrame):
    deplacements_par_horaire = {'night': {}, 'work': {}, 'weekend': {}}
    last_date = {1 : None} # uniquement pour pouvoir le modifier dans une autre fonction
    for index, entry in df_slice_id.iterrows() :
        # Calcule pour chaque position, le temps passé
        date_time = datetime.datetime.fromisoformat(entry['date'])
        gps: tuple = (entry['lat'], entry['long'])
        # En semaine, les POI sont le lieu de travail et la maison
        if date_time.weekday()<5:
            # Entre 22h et 06h -> maison normalement
            if date_time.time()>datetime.time(22,00) or date_time.time()<datetime.time(6,00):
                deplacements_par_horaire['night'][gps] = diff_time(date_time, last_date) if gps not in deplacements_par_horaire['night'] else \
                    deplacements_par_horaire['night'][gps] + diff_time(date_time, last_date)

            elif date_time.time()>datetime.time(9,00) and date_time.time()<datetime.time(16,00):
                deplacements_par_horaire['work'][gps] = diff_time(date_time, last_date) if gps not in deplacements_par_horaire['work'] else \
                    deplacements_par_horaire['work'][gps] + diff_time(date_time, last_date)
        # En weekend, c'est le lieu de divertissement
        else :
            if date_time.time()>datetime.time(10,00) and date_time.time()<datetime.time(18,00):
                deplacements_par_horaire['weekend'][gps] = diff_time(date_time, last_date) if gps not in deplacements_par_horaire['weekend'] else \
                    deplacements_par_horaire['weekend'][gps] + diff_time(date_time, last_date)
    
    # Calcul de la position qui a un maximum de temps passé pour chaque plage horaire
    poi = {'night': None, 'work': None, 'weekend': None}
    for champ in deplacements_par_horaire :
        poi[champ] = max(deplacements_par_horaire[champ], key=lambda x : deplacements_par_horaire[champ][x])
    return deplacements_par_horaire, poi

In [11]:
a = df_slice[df_slice['id']==1].copy()
poi_id1 = computePOI(a)
print(poi_id1[1])
print(poi_id1[0]['night'][poi_id1[1]['night']])
print(poi_id1[0]['work'][poi_id1[1]['work']])
print(poi_id1[0]['weekend'][poi_id1[1]['weekend']])
draw_map(1, df_slice, "Cartes/heatmap_1_poi2", poi_id1[1])

{'night': (45.78, 4.87), 'work': (45.78, 4.88), 'weekend': (45.78, 4.88)}
7 days, 20:23:05
14 days, 5:29:43
8 days, 6:06:53


In [47]:
def shiftHour(entry) ->str:
    originalDate = datetime.datetime.fromisoformat(entry)
    shiftValue = -18 if originalDate.time()>=datetime.time(18,00) else 6
    shiftedDate = originalDate + datetime.timedelta(days=2)
    entry = shiftedDate.strftime("%Y-%m-%d %H:%M:%S")
    return entry

In [52]:
a = df_slice
a['date'] = a['date'].apply(lambda row : shiftHour(row))
poi_id1 = computePOI(a)
print(poi_id1[1])
print(poi_id1[0]['night'][poi_id1[1]['night']])
print(poi_id1[0]['work'][poi_id1[1]['work']])
print(poi_id1[0]['weekend'][poi_id1[1]['weekend']])

C:\Users\Junior\AppData\Local\Temp\ipykernel_69996\1525505104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['date'] = a['date'].apply(lambda row : shiftHour(row))


{'night': (45.78125, 4.87109375), 'work': (45.78125, 4.87109375), 'weekend': (45.78125, 4.87890625)}
5 days, 19:42:10
10 days, 4:57:42
9 days, 12:11:54


In [41]:
df_slice.to_csv("file1.csv", sep="\t", index=False, header=False)

In [ ]:
nuit = poi_id1[1]['night']
df_slice.loc[(df_slice['id']==1) &(df_slice['lat']==nuit[0]) & (df_slice['long']==nuit[1])]['date'] = \
    df_slice.loc[(df_slice['id']==1) &(df_slice['lat']==nuit[0]) & (df_slice['long']==nuit[1])]['date'].apply(lambda row : shiftHour(row))

In [ ]:
poi_id1 = computePOI(df_slice[df_slice['id']==1])
print(poi_id1[1])

In [ ]:
print(poi_id1[0]['night'][poi_id1[1]['night']])
print(poi_id1[0]['work'][poi_id1[1]['work']])
print(poi_id1[0]['weekend'][poi_id1[1]['weekend']])

In [53]:
CHARACTERS_POOL = string.ascii_letters + string.digits

def generator(group_length:int, nb_characters : int):
    pseudo_id_str = ''.join(random.choice(CHARACTERS_POOL) for _ in range(nb_characters))
    pseudo_id_final = m(pseudo_id_str).stringToMorse()
    return[pseudo_id_final]*group_length

def generatePseudoIds(df : pd.DataFrame):
    df['semaine'] = pd.to_datetime(df['date'], format="%Y-%m-%d %H:%M:%S").dt.isocalendar().week
    df['id_x'] = df['id']
    df['id_x'] = df.groupby(['id', 'semaine'])['id_x'].transform(lambda serie : generator(serie.size, nb_characters=5))
    #print(df.groupby(['id', 'semaine'])[['id_x']].first())
    return df[['id_x', 'date', 'long', 'lat']]

b = generatePseudoIds(df_slice.copy())
b.to_csv("anonym1.csv", sep="\t", index=False, header=False)

In [55]:
from metrics import pointsOfInterest, hour, meet, date, distance, tuile
def calculScore(originial_f, anonymise_f):
    metrics = []
    for metric in [pointsOfInterest, hour, meet, date, distance, tuile]:
        metrics.append(metric.main(originial_f, anonymise_f))
    return metrics
calculatedMetrics = calculScore("file1.csv", "anonym1.csv")
score = np.mean(calculatedMetrics)
print(calculatedMetrics, "\n", score)

OrderedDict([((4.87, 45.78), 177077), ((4.88, 45.78), 231400), ((4.95, 45.75), 0), ((4.96, 45.75), 0), ((5.03, 45.75), 0), ((5.04, 45.75), 0), ((5.05, 45.75), 0), ((5.06, 45.75), 1), ((5.93, 44.44), 1), ((4.8, 45.8), 0), ((4.69, 45.78), 0), ((4.83, 45.84), 0), ((4.79, 45.88), 0), ((4.88, 45.88), 74), ((4.86, 45.78), 9474), ((4.88, 45.8), 468), ((4.9, 44.94), 32), ((4.9, 44.9), 43), ((4.9, 44.88), 34), ((4.91, 44.88), 9), ((4.91, 44.84), 48), ((4.91, 44.8), 35), ((4.92, 44.72), 20), ((4.93, 44.72), 28), ((4.93, 44.7), 45), ((4.94, 44.7), 21), ((4.94, 44.66), 15), ((4.95, 44.66), 24), ((4.95, 44.62), 26), ((4.96, 44.62), 36), ((4.96, 44.6), 22), ((4.97, 44.6), 37), ((4.98, 44.6), 19), ((4.98, 44.56), 26), ((4.99, 44.56), 12), ((5.0, 44.56), 0), ((5.0, 44.53), 10), ((5.01, 44.53), 9), ((5.02, 44.53), 0), ((5.02, 44.5), 11), ((5.03, 44.5), 7), ((5.03, 44.47), 0), ((5.04, 44.47), 12), ((5.05, 44.47), 6), ((5.05, 44.44), 5), ((5.06, 44.44), 7), ((5.07, 44.44), 4), ((5.07, 44.4), 6), ((5.08, 